In [1]:
import numpy as np
from ase import Atoms, Atom
from ase.io import write
from ase.data import chemical_symbols
from sklearn import manifold, datasets
from sklearn.decomposition import PCA
from openbabel import pybel as pb
from openbabel import openbabel as ob
from ase.visualize import view

from rdkit.Chem.rdmolfiles import MolFromSmiles,MolToMolFile
from rdkit import Chem
from rdkit.Chem import AllChem

## Some functions to create a 3D structure from a SMILE string

In [12]:
def __ase2xyz__(atoms):
    """
    Prepare a XYZ string from an ASE atoms object.
    """
    # Implementation detail: If PBC should be implemented, the
    # write to xyz needs to be changed to include cell etc.
    atoms.pbc=False
    if any(atoms.get_pbc()):
        raise RuntimeError("Detected PBCs. Not supported (yet)!")
    num_atoms = len(atoms)
    types = atoms.get_chemical_symbols()
    all_atoms = zip(types, atoms.get_positions())
    a_str = str(num_atoms) + "\n" + "\n"
    for atom in all_atoms:
        a_str += atom[0] + " " + " ".join([str(x) for x in atom[1]]) + "\n"
    return a_str


def convert_ase2pybel(atoms):
    """
    Convert an ASE atoms object to pybel (openBabel) molecule.
    The ordering of the Atoms is identical.

    Parameters
    ----------
    atoms : ase.Atoms
        The ASE atoms object

    Returns
    -------
    pymol :
        The pybel molecule.
    """
    atoms.pbc=False
    a_str = __ase2xyz__(atoms)
    pymol = pb.readstring("xyz", a_str)

    return pymol

def convert_ase2rdkit(atoms, removeHs=False):
    """
    Convert an ASE atoms object to rdkit molecule.
    The ordering of the Atoms is identical.


    Important: Implemented only for clusters, not PBC!
    rdkit does not keep xyz coordinates, therefore
    a backconversion is not possible yet.

    Parameters
    ----------
    atoms : ase.Atoms
        The ASE atoms object
    removeHs : Bool
        If True, remove all H atoms from molecule.

    Returns
    -------
    mol : rdkit.Chem.rdchem.Mol
        The rdkit molecule object.
    """
    a_str = __ase2xyz__(atoms)
    pymol = pb.readstring("xyz", a_str)
    mol = pymol.write("mol")
    mol = Chem.MolFromMolBlock(mol, removeHs=removeHs)
    return mol

def pybel2ase(mol):  
    asemol = Atoms()
    species=[chemical_symbols[atm.atomicnum] for atm in mol.atoms]
    pos=np.asarray([atm.coords for atm in mol.atoms])
    pca = PCA(n_components=3)
    posnew=pca.fit_transform(pos)
    atoms = Atoms(species, positions=posnew)
    sys_size = np.ptp(atoms.positions,axis=0)
    atoms.pbc=True
    atoms.cell = sys_size + 10
    atoms.center()
    
    return atoms

def rdkit2ase(m):
    pos = m.GetConformer().GetPositions()
    natoms = m.GetNumAtoms()
    species = [m.GetAtomWithIdx(j).GetSymbol() for j in range(natoms)]                
#Get the principal axes and realign the molecule
    pca = PCA(n_components=3)
    pca.fit(pos)
    posnew=pca.transform(pos)        
#Set the z to 0.0       
    #posnew[:,2]=0.0
    atoms = Atoms(species, positions=posnew)  
    sys_size = np.ptp(atoms.positions,axis=0)
    atoms.pbc=True
    atoms.cell = sys_size + 10
    atoms.center()
        
    return atoms


def pybel_opt(smile,steps):
    obconversion = ob.OBConversion()
    obconversion.SetInFormat('smi')
    obmol = ob.OBMol()
    obconversion.ReadString(obmol,smile)
    
    pbmol = pb.Molecule(obmol)
    pbmol.make3D(forcefield="uff", steps=50)
    
    pbmol.localopt(forcefield="gaff", steps=200)
    pbmol.localopt(forcefield="mmff94", steps=100)
    
    f_f = pb._forcefields["uff"]
    f_f.Setup(pbmol.OBMol)
    f_f.ConjugateGradients(steps, 1.0e-9)
    f_f.GetCoordinates(pbmol.OBMol)
    #print(f_f.Energy(), f_f.GetUnit())
    
    return pybel2ase(pbmol)

def try_rdkit(smile):
    test = Chem.MolFromSmiles(smile)
    test = Chem.AddHs(test)
    
    #test=convert_ase2rdkit(pybel_opt(smile,10))
    
    return AllChem.EmbedMolecule(test, maxAttempts=10, randomSeed=42)
    
def rdkit_opt(smile,steps):
    m = Chem.MolFromSmiles(smile)
    m = Chem.AddHs(m)
    
    # adjust possible problems with hydroges
    component = Chem.MolToSmiles(m, canonical=False)
    component = component.replace("[", "").replace("]","")
    m = Chem.MolFromSmiles(component)
    m = Chem.AddHs(m)
    
    # generate the 3D coordinates
    AllChem.EmbedMolecule(m, maxAttempts=20, randomSeed=42)
    AllChem.UFFOptimizeMolecule(m,maxIters=steps)
    
    return rdkit2ase(m)

def mol_from_smiles(smile,steps=10):
    if try_rdkit(smile)==0:
        #print("Going for rdkit")
        return rdkit_opt(smile,steps)
    else:
        #print("Going for pybel")
        return pybel_opt(smile,steps)

In [13]:
# Example smiles

smi="C1(C(C(C2=C(C1=CC=C3C4=CC=C5)C3=C6C7=C2C8=CC9=C7C%10=C%11C(C%12=C%13C%14=C%15C(C(C%16=C%17C(C%18=C%19C%20=C%21C(C(C%22=CC=CC%23=C%22C%21=C(C%19=C%17C%24=C%25)C%25=C%23)=CC=C%26%27)=C%26C%28=C%20C(C(C(C%28=C(C%27=C/%29)NC%29=C/%30)=N/%31)=CC%31=C\C%32=CC=C(/C=C%33C=CC%30=N/%33)N%32)=C%18)=CC%34=C%16C%15=C(C%13=C%11C9=C%35)C%35=C%34)=C%24C=C%36%37)=C%36C%38=C%14C(C(C(C%38=C(C%37=C/%39)NC%39=C/%40)=N/%41)=CC%41=C\C%42=CC=C(/C=C%43C=CC%40=N/%43)N%42)=C%12)=CC%44=C%10C6=C4C5=C%44)=C(C8=C/%45)NC%45=C/%46)=N/%47)=CC%47=C\C%48=CC=C(/C=C%49C=CC%46=N/%49)N%48"
smi2="C12=CC=CC3=C1C(CC4=C3C5=C(C(C(C6=CC=CC7=C6C(C8)=CC=C7)=C8C=C9)=C9C=C%10)C%10=CC=C5C=C4)=CC=C2"
smi1="C=C1C(C=C(C=C2)C3=CC=C(C4=CC(C=C5)=C(C6=C5C=C(C7=CC=C(C8=CC9=C(C%10=C(C9=C)C=CC=C%10)C=C8C%11=C)C%11=C7)C=C6)C=C4C=C%12)C%12=C3)=C2C(C=C%13%14)=C1C=C%13C%15=CC=CC=C%15C%14=C"

In [20]:
from ase.visualize import view
mol=mol_from_smiles(smi)
#view(mol)

## ADD GOLD

The following cell is just to place the molecule on top of a predefined Gold slab. Skip it if you don't need it or improve if you need a better slab

In [ ]:
smimol=mol_from_smiles(smi2)
from ase.io import read
gold_slab=read("examples/Au896.xyz")

# get cm gol
cm_gold=gold_slab.get_center_of_mass()
# flatten the mol
#smimol.positions[:,2]=0
# get mol cm
cm_mol=smimol.get_center_of_mass()
# get the diff
diff=cm_gold-cm_mol
# shit the mol
smimol.positions+=diff
# shift on top
smimol.positions[:,2]+=(max(gold_slab.positions[:,2])-smimol.get_center_of_mass()[2])+3.2

# join the slab and the mol

mol = Atoms()
for atm in gold_slab:
    mol.append(atm)
for atm in smimol:
    mol.append(atm)

In [ ]:
write("mol-rdkit.pdb",mol)

## OPTIMIZE THE MOLECULE USING XTB ( or other semiempirical methods)

In [22]:
from ase.build import molecule
from xtb.ase.calculator import XTB

In [23]:
mol.pbc=False
#newstrxtb.calc=XTB(method="GFN2-xTB")
# GFNFF is very fast. See here for other methods:
# https://xtb-python.readthedocs.io/en/latest/general-api.html#available-calculation-methods
mol.calc=XTB(method="GFNFF")

Evaluate the following if you want to fix some atoms, e.g. the metal substrate.

In [ ]:
from ase.constraints import FixAtoms

fix_z=max(mol[np.where(np.asarray(mol.get_chemical_symbols())=="Au")[0]].positions[:,2])
idatoms_tobefixed = np.where(mol.positions[:,2]<fix_z)[0]
atoms_tobefixed = FixAtoms(idatoms_tobefixed)
mol.set_constraint(atoms_tobefixed)

In [24]:
from ase.optimize.lbfgs import LBFGS

hareV=27.211399

opt = LBFGS(mol)
# Remove logfile=none to see an output duri
opt.run(fmax=0.00001,logfile=None)
forces = mol.get_forces()
energy = mol.get_potential_energy()

#
print("Final energy is", energy, "eV")
print("Final energy is", energy/hareV, "hartree")

#print("All done, final positions are:")
#print(positions)

       Step     Time          Energy         fmax
*Force-consistent energies used in optimization.
LBFGS:    0 15:14:18    -1399.201218*       7.4569
LBFGS:    1 15:14:18    -1402.378861*       4.3503
LBFGS:    2 15:14:18    -1404.195239*       2.8526
LBFGS:    3 15:14:18    -1405.268375*       1.7804
LBFGS:    4 15:14:18    -1406.039074*       1.2273
LBFGS:    5 15:14:18    -1406.656252*       1.1812
LBFGS:    6 15:14:18    -1407.170157*       1.0347
LBFGS:    7 15:14:18    -1407.611583*       0.7832
LBFGS:    8 15:14:18    -1407.963547*       0.9046
LBFGS:    9 15:14:19    -1408.239394*       0.9429
LBFGS:   10 15:14:19    -1408.468294*       0.8303
LBFGS:   11 15:14:19    -1408.655689*       0.5293
LBFGS:   12 15:14:19    -1408.827700*       0.4764
LBFGS:   13 15:14:19    -1408.996069*       0.5981
LBFGS:   14 15:14:19    -1409.150414*       0.6825
LBFGS:   15 15:14:19    -1409.291936*       0.6173
LBFGS:   16 15:14:19    -1409.420711*       0.5415
LBFGS:   17 15:14:19    -1409.5394

LBFGS:  159 15:14:22    -1413.357189*       0.0825
LBFGS:  160 15:14:22    -1413.365508*       0.1003
LBFGS:  161 15:14:22    -1413.372187*       0.0840
LBFGS:  162 15:14:22    -1413.381764*       0.0829
LBFGS:  163 15:14:22    -1413.387159*       0.0741
LBFGS:  164 15:14:22    -1413.396797*       0.1155
LBFGS:  165 15:14:22    -1413.403557*       0.1145
LBFGS:  166 15:14:22    -1413.413298*       0.1183
LBFGS:  167 15:14:22    -1413.420207*       0.1031
LBFGS:  168 15:14:22    -1413.428824*       0.0858
LBFGS:  169 15:14:22    -1413.435184*       0.0757
LBFGS:  170 15:14:22    -1413.443524*       0.0945
LBFGS:  171 15:14:22    -1413.450038*       0.0819
LBFGS:  172 15:14:22    -1413.456872*       0.0739
LBFGS:  173 15:14:22    -1413.462812*       0.0762
LBFGS:  174 15:14:22    -1413.468833*       0.0737
LBFGS:  175 15:14:22    -1413.474500*       0.0781
LBFGS:  176 15:14:23    -1413.480074*       0.0672
LBFGS:  177 15:14:23    -1413.485631*       0.0720
LBFGS:  178 15:14:23    -1413.4

LBFGS:  320 15:14:25    -1413.733089*       0.0200
LBFGS:  321 15:14:25    -1413.733571*       0.0181
LBFGS:  322 15:14:25    -1413.734033*       0.0184
LBFGS:  323 15:14:25    -1413.734464*       0.0179
LBFGS:  324 15:14:25    -1413.734863*       0.0179
LBFGS:  325 15:14:25    -1413.735251*       0.0179
LBFGS:  326 15:14:25    -1413.735663*       0.0197
LBFGS:  327 15:14:25    -1413.736112*       0.0224
LBFGS:  328 15:14:25    -1413.736582*       0.0187
LBFGS:  329 15:14:25    -1413.737051*       0.0190
LBFGS:  330 15:14:25    -1413.737527*       0.0188
LBFGS:  331 15:14:25    -1413.738026*       0.0204
LBFGS:  332 15:14:25    -1413.738551*       0.0204
LBFGS:  333 15:14:25    -1413.739091*       0.0224
LBFGS:  334 15:14:25    -1413.739655*       0.0253
LBFGS:  335 15:14:25    -1413.740267*       0.0289
LBFGS:  336 15:14:25    -1413.740919*       0.0259
LBFGS:  337 15:14:25    -1413.741555*       0.0246
LBFGS:  338 15:14:25    -1413.742140*       0.0250
LBFGS:  339 15:14:25    -1413.7

LBFGS:  481 15:14:28    -1413.854301*       0.0204
LBFGS:  482 15:14:28    -1413.854805*       0.0192
LBFGS:  483 15:14:28    -1413.855348*       0.0262
LBFGS:  484 15:14:28    -1413.855914*       0.0260
LBFGS:  485 15:14:28    -1413.856484*       0.0267
LBFGS:  486 15:14:28    -1413.857049*       0.0255
LBFGS:  487 15:14:28    -1413.857593*       0.0249
LBFGS:  488 15:14:28    -1413.858094*       0.0235
LBFGS:  489 15:14:28    -1413.858557*       0.0192
LBFGS:  490 15:14:28    -1413.859004*       0.0189
LBFGS:  491 15:14:28    -1413.859449*       0.0221
LBFGS:  492 15:14:28    -1413.859886*       0.0211
LBFGS:  493 15:14:28    -1413.860305*       0.0209
LBFGS:  494 15:14:28    -1413.860700*       0.0197
LBFGS:  495 15:14:29    -1413.861074*       0.0203
LBFGS:  496 15:14:29    -1413.861428*       0.0196
LBFGS:  497 15:14:29    -1413.861772*       0.0194
LBFGS:  498 15:14:29    -1413.862122*       0.0223
LBFGS:  499 15:14:29    -1413.862483*       0.0238
LBFGS:  500 15:14:29    -1413.8

LBFGS:  642 15:14:31    -1413.887812*       0.0096
LBFGS:  643 15:14:31    -1413.887912*       0.0088
LBFGS:  644 15:14:31    -1413.888009*       0.0094
LBFGS:  645 15:14:31    -1413.888100*       0.0107
LBFGS:  646 15:14:31    -1413.888184*       0.0104
LBFGS:  647 15:14:31    -1413.888263*       0.0093
LBFGS:  648 15:14:31    -1413.888336*       0.0090
LBFGS:  649 15:14:31    -1413.888404*       0.0081
LBFGS:  650 15:14:31    -1413.888465*       0.0074
LBFGS:  651 15:14:31    -1413.888519*       0.0080
LBFGS:  652 15:14:31    -1413.888574*       0.0085
LBFGS:  653 15:14:31    -1413.888634*       0.0080
LBFGS:  654 15:14:31    -1413.888701*       0.0079
LBFGS:  655 15:14:31    -1413.888770*       0.0081
LBFGS:  656 15:14:31    -1413.888836*       0.0074
LBFGS:  657 15:14:31    -1413.888901*       0.0077
LBFGS:  658 15:14:31    -1413.888969*       0.0089
LBFGS:  659 15:14:31    -1413.889041*       0.0085
LBFGS:  660 15:14:31    -1413.889113*       0.0076
LBFGS:  661 15:14:31    -1413.8

...

LBFGS:  1755 15:14:50    -1413.903476*       0.0002
LBFGS:  1756 15:14:50    -1413.903476*       0.0001
LBFGS:  1757 15:14:50    -1413.903476*       0.0001
LBFGS:  1758 15:14:50    -1413.903476*       0.0001
LBFGS:  1759 15:14:50    -1413.903476*       0.0001
LBFGS:  1760 15:14:50    -1413.903476*       0.0001
LBFGS:  1761 15:14:50    -1413.903476*       0.0001
LBFGS:  1762 15:14:50    -1413.903476*       0.0001
LBFGS:  1763 15:14:50    -1413.903476*       0.0001
LBFGS:  1764 15:14:50    -1413.903476*       0.0002
LBFGS:  1765 15:14:50    -1413.903476*       0.0002
LBFGS:  1766 15:14:50    -1413.903476*       0.0001
LBFGS:  1767 15:14:50    -1413.903476*       0.0001
LBFGS:  1768 15:14:50    -1413.903476*       0.0001
LBFGS:  1769 15:14:50    -1413.903476*       0.0001
LBFGS:  1770 15:14:50    -1413.903476*       0.0001
LBFGS:  1771 15:14:50    -1413.903476*       0.0001
LBFGS:  1772 15:14:50    -1413.903476*       0.0001
LBFGS:  1773 15:14:50    -1413.903476*       0.0001
LBFGS:  1774

LBFGS:  1913 15:14:52    -1413.903479*       0.0001
LBFGS:  1914 15:14:52    -1413.903479*       0.0001
LBFGS:  1915 15:14:52    -1413.903479*       0.0001
LBFGS:  1916 15:14:52    -1413.903479*       0.0001
LBFGS:  1917 15:14:52    -1413.903479*       0.0001
LBFGS:  1918 15:14:52    -1413.903479*       0.0001
LBFGS:  1919 15:14:52    -1413.903479*       0.0001
LBFGS:  1920 15:14:52    -1413.903479*       0.0001
LBFGS:  1921 15:14:52    -1413.903479*       0.0001
LBFGS:  1922 15:14:52    -1413.903479*       0.0001
LBFGS:  1923 15:14:52    -1413.903479*       0.0001
LBFGS:  1924 15:14:52    -1413.903479*       0.0001
LBFGS:  1925 15:14:52    -1413.903479*       0.0001
LBFGS:  1926 15:14:52    -1413.903479*       0.0001
LBFGS:  1927 15:14:52    -1413.903479*       0.0001
LBFGS:  1928 15:14:52    -1413.903479*       0.0001
LBFGS:  1929 15:14:52    -1413.903479*       0.0001
LBFGS:  1930 15:14:52    -1413.903479*       0.0001
LBFGS:  1931 15:14:53    -1413.903479*       0.0001
LBFGS:  1932

LBFGS:  2071 15:14:55    -1413.903480*       0.0001
LBFGS:  2072 15:14:55    -1413.903480*       0.0001
LBFGS:  2073 15:14:55    -1413.903480*       0.0001
LBFGS:  2074 15:14:55    -1413.903480*       0.0001
LBFGS:  2075 15:14:55    -1413.903480*       0.0000
LBFGS:  2076 15:14:55    -1413.903480*       0.0000
LBFGS:  2077 15:14:55    -1413.903480*       0.0000
LBFGS:  2078 15:14:55    -1413.903480*       0.0001
LBFGS:  2079 15:14:55    -1413.903480*       0.0001
LBFGS:  2080 15:14:55    -1413.903480*       0.0001
LBFGS:  2081 15:14:55    -1413.903480*       0.0001
LBFGS:  2082 15:14:55    -1413.903480*       0.0001
LBFGS:  2083 15:14:55    -1413.903480*       0.0001
LBFGS:  2084 15:14:55    -1413.903480*       0.0001
LBFGS:  2085 15:14:55    -1413.903480*       0.0000
LBFGS:  2086 15:14:55    -1413.903480*       0.0001
LBFGS:  2087 15:14:55    -1413.903480*       0.0000
LBFGS:  2088 15:14:55    -1413.903480*       0.0000
LBFGS:  2089 15:14:55    -1413.903480*       0.0001
LBFGS:  2090

LBFGS:  2229 15:14:58    -1413.903481*       0.0001
LBFGS:  2230 15:14:58    -1413.903481*       0.0000
LBFGS:  2231 15:14:58    -1413.903481*       0.0001
LBFGS:  2232 15:14:58    -1413.903481*       0.0001
LBFGS:  2233 15:14:58    -1413.903481*       0.0001
LBFGS:  2234 15:14:58    -1413.903481*       0.0001
LBFGS:  2235 15:14:58    -1413.903481*       0.0001
LBFGS:  2236 15:14:58    -1413.903481*       0.0000
LBFGS:  2237 15:14:58    -1413.903481*       0.0000
LBFGS:  2238 15:14:58    -1413.903481*       0.0000
LBFGS:  2239 15:14:58    -1413.903481*       0.0001
LBFGS:  2240 15:14:58    -1413.903481*       0.0001
LBFGS:  2241 15:14:58    -1413.903481*       0.0001
LBFGS:  2242 15:14:58    -1413.903481*       0.0000
LBFGS:  2243 15:14:58    -1413.903481*       0.0001
LBFGS:  2244 15:14:58    -1413.903481*       0.0000
LBFGS:  2245 15:14:58    -1413.903481*       0.0000
LBFGS:  2246 15:14:58    -1413.903481*       0.0000
LBFGS:  2247 15:14:58    -1413.903481*       0.0001
LBFGS:  2248

LBFGS:  2387 15:15:00    -1413.903482*       0.0000
LBFGS:  2388 15:15:00    -1413.903482*       0.0000
LBFGS:  2389 15:15:00    -1413.903482*       0.0000
LBFGS:  2390 15:15:00    -1413.903482*       0.0000
LBFGS:  2391 15:15:00    -1413.903482*       0.0000
LBFGS:  2392 15:15:00    -1413.903482*       0.0000
LBFGS:  2393 15:15:00    -1413.903482*       0.0000
LBFGS:  2394 15:15:00    -1413.903482*       0.0000
LBFGS:  2395 15:15:00    -1413.903482*       0.0000
LBFGS:  2396 15:15:00    -1413.903482*       0.0000
LBFGS:  2397 15:15:00    -1413.903482*       0.0000
LBFGS:  2398 15:15:00    -1413.903482*       0.0000
LBFGS:  2399 15:15:00    -1413.903482*       0.0000
LBFGS:  2400 15:15:00    -1413.903482*       0.0000
LBFGS:  2401 15:15:00    -1413.903482*       0.0000
LBFGS:  2402 15:15:01    -1413.903482*       0.0000
LBFGS:  2403 15:15:01    -1413.903482*       0.0000
LBFGS:  2404 15:15:01    -1413.903482*       0.0000
LBFGS:  2405 15:15:01    -1413.903482*       0.0000
LBFGS:  2406

In [25]:
view(mol)

In [ ]:
write("mol-optimized.pdb",mol)

## Compute energy and forces using DFT and Beyond-DFT

In [19]:
from ase.calculators.psi4 import Psi4
from ase.build import molecule
import numpy as np

In [ ]:
atoms = molecule('H2O')

#atoms = Atoms(mol.get_chemical_symbols(), positions=mol.positions)

# See here for the methods:
# http://www.psicode.org/psi4manual/master/autodoc_dft_energy.html#table-energy-dft

# here the basisets:
# http://www.psicode.org/psi4manual/master/basissets_byfamily.html


# and check also this link for a nice overview of the Psi4 calculator in ASE:
# https://gitlab.com/ase/ase/blob/master/doc/ase/calculators/psi4.rst
calc = Psi4(atoms = atoms,
        method = 'b3lyp', #'b3lyp'
        memory = '16000MB', # the default is 500MB, be aware!
        basis = '6-311g_d_p_',
        num_threads="max"
           )

atoms.calc = calc
print(atoms.get_potential_energy())
print(atoms.get_forces())
